In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
import ast

In [4]:
df= pd.read_csv("train.csv")

In [5]:
print(df.describe())

       untagged_sentence tagged_sentence
count              47340           47340
unique             46610           46637
top                ['.']    [(')', ')')]
freq                  52              46


In [6]:
print(df.head(n=10))

                                   untagged_sentence  \
0  ['The', 'jury', 'further', 'said', 'in', 'term...   
1  ['The', 'September-October', 'term', 'jury', '...   
2  ['``', 'Only', 'a', 'relative', 'handful', 'of...   
3  ['The', 'jury', 'said', 'it', 'did', 'find', '...   
4  ['It', 'recommended', 'that', 'Fulton', 'legis...   
5  ['The', 'grand', 'jury', 'commented', 'on', 'a...   
6                             ['Merger', 'proposed']   
7  ['However', ',', 'the', 'jury', 'said', 'it', ...   
8  ['The', 'City', 'Purchasing', 'Department', ',...   
9  ['It', 'urged', 'that', 'the', 'city', '``', '...   

                                     tagged_sentence  
0  [('The', 'AT'), ('jury', 'NN'), ('further', 'R...  
1  [('The', 'AT'), ('September-October', 'NP'), (...  
2  [('``', '``'), ('Only', 'RB'), ('a', 'AT'), ('...  
3  [('The', 'AT'), ('jury', 'NN'), ('said', 'VB')...  
4  [('It', 'PP'), ('recommended', 'VB'), ('that',...  
5  [('The', 'AT'), ('grand', 'JJ'), ('jury', 'NN'... 

In [7]:
data = []
for index, row in tqdm(df.iterrows()):
    data.append(ast.literal_eval(row['tagged_sentence']))

47340it [00:08, 5420.11it/s]


In [6]:
# data[0]
dict_train_tokens={}
dict_train_tags={}
len_of_data= len(data)
for lists in data:
    for tuples in lists:
        token= tuples[0]
        tag= tuples[1]
    
        if token in dict_train_tokens:
            dict_train_tokens[token]+=1
        else:
            dict_train_tokens[token]=1
            
        if tag in dict_train_tags:
            dict_train_tags[tag]+=1
        else:
            dict_train_tags[tag]=1
        
             


In [7]:
len_train_tags=len(dict_train_tags)
len_train_tokens=len(dict_train_tokens)
print(len_train_tags)

49


In [8]:
dict_index_to_tags={}
dict_tags_to_index={}

idx=0

for tag in dict_train_tags:
    if idx not in dict_index_to_tags:
        dict_index_to_tags[idx]= tag 
        dict_tags_to_index[tag]= idx 
    idx+=1
    

In [9]:

print(idx)

49


In [10]:
dict_index_to_tokens={}
dict_tokens_to_index={}
idx=0

for token in dict_train_tokens:
    if idx not in dict_index_to_tokens:
        dict_index_to_tokens[idx]= token
        dict_tokens_to_index[token]=idx
    idx+=1

In [12]:
#Emission Matrix
emission_prob_matrix= np.zeros(( len_train_tags,len_train_tokens))
for lists in data:
    for tuples in lists:
        token= tuples[0]
        tag= tuples[1]
        
        emission_prob_matrix[dict_tags_to_index[tag]][dict_tokens_to_index[token]]+=1
        
        
print(emission_prob_matrix.shape)


row_sums = emission_prob_matrix.sum(axis=1)

for i, row_sum in enumerate(row_sums):
    if row_sum > 0:
        emission_prob_matrix[i,:] /= row_sum

print(np.sum(emission_prob_matrix[2,:]))

(49, 51208)
0.9999999999999999


In [13]:
#Transition Matrix
transition_prob_matrix= np.zeros((len_train_tags+1, len_train_tags+1))

for list in data:
    len_list= len(list)
    for tuple_index in range(len_list-1):
        first_tuple_tag= list[tuple_index][1] 
        second_tuple_tag= list[tuple_index+1][1]
        if tuple_index==0:
            transition_prob_matrix[len_train_tags][dict_tags_to_index[first_tuple_tag]]+=1
            
        elif tuple_index==len_list-1-1:
            transition_prob_matrix[dict_tags_to_index[second_tuple_tag]][len_train_tags]+=1
        
        
        transition_prob_matrix[dict_tags_to_index[first_tuple_tag]][dict_tags_to_index[second_tuple_tag]]+=1
            
            
print(transition_prob_matrix.shape)  
row_sums = transition_prob_matrix.sum(axis=1)
for i, row_sum in enumerate(row_sums):
    if row_sum > 0:
        transition_prob_matrix[i, :] /= row_sum
print(np.sum(transition_prob_matrix[0,:]))
            

            
    


(50, 50)
1.0


In [14]:
print(emission_prob_matrix.shape)
print(transition_prob_matrix.shape)

(49, 51208)
(50, 50)


In [15]:
len_train_tags

49

In [16]:
df= pd.read_csv("test_small.csv")

In [17]:

test_data_dict = pd.Series(df.untagged_sentence.values,index=df.id).to_dict()

In [56]:
def viterbi_algorithm_new(sentence, transition_prob_matrix,emission_prob_matrix):
    prob_matrix= np.zeros((len_train_tags, len_train_tokens))
    m,n= prob_matrix.shape
    len_of_sentence=len(sentence)
    bp= np.zeros((m,n))
    all_tages=[]
    for i in range(len(sentence)):
        for j in range(m):
            
            if sentence[i] not in dict_train_tokens:
                emission_prob=1
            else:
                emission_prob=emission_prob_matrix[j][dict_tokens_to_index[sentence[i]]]
            if(i==0):
                prob_matrix[j][i]=transition_prob_matrix[len_train_tags][j]*emission_prob
            else:
                prev_col= prob_matrix[:,i-1].reshape(1,-1).copy()
                for k in range(m):
                    prev_col[0][k]*=(transition_prob_matrix[k][j]*emission_prob)
                maxpos=np.argmax(prev_col)
                prob_matrix[j][i]=prev_col[0][maxpos]
                bp[j][i]=maxpos
    last_tag_index=np.argmax(prob_matrix[:,-1])
    all_tages.append(last_tag_index)

    for i in range(len_of_sentence-1,0,-1):
        all_tages.insert(0,int(bp[all_tages[0],i]))
    tagged_sentence=[]
    for i in range(len_of_sentence):
        tagged_sentence.append((sentence[i],dict_index_to_tags[all_tages[i]]))
    return tagged_sentence

    

In [57]:
def get_predictions(test_data_dict):
    data_list=[]
    predictions={}
    ctr=0
    for id in test_data_dict:
        data_list.append((id,test_data_dict[id]))
        ctr+=1
    for data_row in data_list:
        print(data_row[0])
        temp=ast.literal_eval(i[1])
        tagged_sentence=viterbi_algorithm_new(temp,transition_prob_matrix,emission_prob_matrix)
        
        predictions[data_row[0]]= tagged_sentence
    return predictions

In [58]:
test_predictions=get_predictions(test_data_dict)

3
4
6
12
15
16
18
19
27
28
32
36
37
44
45
48
50
52
56
58
60
61
65
68
69
70
71
72
77
78
79
80
82
84
85
86
90
91
94
95
97
100
101
105
107
109
112
114
115
117
118
119
122
124
133
135
136
140
143
144
145
147
151
158
160
161
162
163
164
168
172
175
178
188
189
191
194
198
201
204
205
206
208
209
212
213
214
215
216
217
218
220
222
228
229
232
234
237
238
239
241
243
244
246
251
252
256
259
260
263
266
268
272
274
276
279
280
283
285
288
289
291
296
297
299
301
302
307
308
312
315
316
317
320
324
326
327
328
331
332
337
338
339
341
342
343
344
350
351
354
360
361
363
367
368
369
371
372
373
376
377
383
384
389
391
392
401
402
404
405
406
407
409
410
413
414
420
422
424
425
427
429
432
433
434
437
438
443
444
446
447
449
450
453
454
455
456
458
462
463
467
468
471
473
476
478
479
481
482
485
487
488
492
494
499
503
504
510
511
513
515
516
519
522
523
531
532
537
538
540
545
547
551
553
558
560
561
562
565
566
567
570
572
573
576
577
578
579
580
583
586
588
591
592
596
597
605
606
607
612
616


In [59]:
predictions_df = pd.DataFrame(test_predictions,columns=['id', 'tagged_sentence'])
output_csv_path = 'predictions_hmm2.csv'
predictions_df.to_csv(output_csv_path, index=False)

In [60]:
import csv


csv_file_path = 'predictions_hmm2.csv'


with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['id', 'tagged_sentence'])

    for key, values in test_predictions.items():
        writer.writerow([key, values])


csv_file_path


'predictions_hmm2.csv'